# Introduction


Thanks to https://www.kaggle.com/kenseitrg/simple-fastai-exercise Please upvote!

The fastai library simplifies training fast and accurate neural nets using modern best practices. “fast.ai” offers a practical way of mastering Deep Learning by straight away going coding and implementing of real Kaggle competitions. Then later slowly it builds a solid understanding about the underlying concepts of ‘Neural Networks’, how these concepts can be applied to real world scenarios, their(Neural Networks) limitations.

Here is the link to the courses (free and no ads) : https://www.fast.ai/


### The code has been commented as comprehensively as possible for explanation purpose.

#### Here is a graph of the key module dependencies

![Here is a graph of the key module dependencies](https://docs.fast.ai/imgs/dependencies.svg)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
from pathlib import Path
from fastai import *
from fastai.vision import *
import torch

In [ ]:
data_folder = Path("../input")

In [ ]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/sample_submission.csv")

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
## Reads images from the test folder with the dataframe that contains ids that act as the image name for test
test_img = ImageList.from_df(test_df, path=data_folder/'test', folder='test')


## setting the transformations to be applied on the images 
transformations = get_transforms(do_flip=True, flip_vert=True, max_rotate=10.0, max_zoom=1, max_lighting=0.1, 
                                 max_warp=0.2, p_affine=0.75, p_lighting=0.75)



##########
## (ImageList.from_df(train_df, path=data_folder/'train', folder='train') gives us ImageList object to apply 

# split_by_rand_pct(): Split the items randomly by putting valid_pct in the validation set, optional seed can be passed.
# add_test() : Add test set containing items with an arbitrary label (here arbitrary label is 0.5)
# label_from_df(): Label self.items from the values in cols in self.inner_df.
# transform() : to run the transformations decides in the transfomations object above
# databunch() : to convert to a DataBunch bs is batch size

# torch.device() argument ->
#----------------------------------------------------------------------------------------------------------------------
# A torch.device is an object representing the device on which a torch.Tensor is or will be allocated.

# The torch.device contains a device type ('cpu' or 'cuda') and optional device ordinal for the device type. 
# If the device ordinal is not present, this represents the current device for the device type; e.g. a torch.
# Tensor constructed with device 'cuda' is equivalent to 'cuda:X' where X is the result of torch.cuda.current_device().

# A torch.Tensor’s device can be accessed via the Tensor.device property.

# A torch.device can be constructed via a string or via a string and device ordinal

#----------------------------------------------------------------------------------------------------------------------

train_image_list_bunch = (ImageList.from_df(train_df, path=data_folder/'train', folder='train')
        .split_by_rand_pct(0.02)
        .label_from_df()
        .add_test(test_img)
        .transform(transformations, size=128)
        .databunch(path='.', bs=64, device= torch.device('cuda:0'))
        .normalize(imagenet_stats)
       )

In [ ]:
train_image_list_bunch.show_batch(rows=4, figsize=(7,6))

In [ ]:
# The cnn_learner factory method helps you to automatically get a pretrained model from a given architecture 
# with a custom head that is suitable for your data.

# cnn_learner[source][test]
# cnn_learner(data:DataBunch, base_arch:Callable, cut:Union[int, Callable]=None, pretrained:bool=True, 
#             lin_ftrs:Optional[Collection[int]]=None, ps:Floats=0.5, custom_head:Optional[Module]=None, 
#             split_on:Union[Callable, Collection[ModuleList], NoneType]=None, bn_final:bool=False, 
#             init='kaiming_normal_', concat_pool:bool=True, **kwargs:Any) → Learner

# Build convnet style learner.

# This method creates a Learner object from the data object and model inferred from it with the backbone given in arch. 
# Specifically, it will cut the model defined by arch (randomly initialized if pretrained is False) at the last 
# convolutional layer by default

learn = cnn_learner(train_image_list_bunch, models.densenet161, metrics=[error_rate, accuracy])

In [ ]:
# lr_find(): Explore lr from start_lr to end_lr over num_it iterations in learn. If stop_div, 
# stops when loss diverges.

learn.lr_find()
learn.recorder.plot()

In [ ]:
## fit_one_cycle(): Fits number of cycles/epochs as per arguments


# fit_one_cycle(learn:Learner, cyc_len:int, max_lr:Union[float, Collection[float], slice]=slice(None, 0.003, None), 
#               moms:Point=(0.95, 0.85), div_factor:float=25.0, pct_start:float=0.3, final_div:float=None, 
#               wd:float=None, callbacks:Optional[Collection[Callback]]=None, tot_epochs:int=None, 
#               start_epoch:int=None)

# Fit a model following the 1cycle policy.


lr = 1e-01
learn.fit_one_cycle(5, slice(lr))

In [ ]:
#get pedictions from the model
preds,_ = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
test_df.has_cactus = preds.numpy()[:, 0]

In [ ]:
test_df.to_csv('submission.csv', index=False)